# Task Status Checker Demo

This notebook demonstrates the TaskStatusChecker, which is a deterministic function (no LLM calls) that:
1. Analyzes the current query tree state
2. Determines what action should be taken next
3. Automatically updates the current node pointer for processing
4. Provides detailed status reports and recommendations

## Key Features
- **Deterministic analysis** - no LLM calls, purely rule-based
- **Tree traversal logic** - processes leaf nodes first, then parents
- **Quality assessment** - distinguishes between good and poor results
- **Current node management** - automatically selects next node to process
- **Detailed reporting** - shows tree state, node content, and recommendations

**Important**: Run all cells in order from the beginning. The cells depend on variables defined in previous cells.

In [1]:
import os
import sys
import asyncio
import time
import json
from pathlib import Path
from typing import Dict, Any, List
from dotenv import load_dotenv

sys.path.append('../src')
load_dotenv()

# Import required components
from keyvalue_memory import KeyValueMemory
from task_status_checker import TaskStatusChecker, TaskStatusCheckerArgs
from query_tree_manager import QueryTreeManager
from node_history_manager import NodeHistoryManager
from task_context_manager import TaskContextManager
from memory_content_types import QueryNode, NodeStatus, TaskStatus, ExecutionResult

print("✅ All imports successful!")
print(f"Available modules: TaskStatusChecker, QueryTreeManager, KeyValueMemory")
print(f"Memory content types: QueryNode, NodeStatus, TaskStatus, ExecutionResult")

✅ All imports successful!
Available modules: TaskStatusChecker, QueryTreeManager, KeyValueMemory
Memory content types: QueryNode, NodeStatus, TaskStatus, ExecutionResult


## 1. Setup Memory and Initialize Task Context

First, let's set up the memory and initialize a basic task context (like the orchestrator would do).

In [2]:
# Initialize shared memory and managers
memory = KeyValueMemory()

# Initialize managers (like orchestrator does)
tree_manager = QueryTreeManager(memory)
task_checker = TaskStatusChecker(memory)
task_manager = TaskContextManager(memory)

# Initialize task context
await task_manager.initialize("status-check-demo", "Find all students", "demo_database")

# Create a simple tree (clear happens automatically in initialize)
root_id = await tree_manager.initialize("Find all students")

print(f"✅ Setup complete!")
print(f"Task context initialized with query: 'Find all students'")
print(f"Query tree root node created: {root_id}")

# Check initial status
args = TaskStatusCheckerArgs(task="Check initial single node")
result = await task_checker.run(args)
print(f"\n📊 Initial Status Check:")
print(result)

✅ Setup complete!
Task context initialized with query: 'Find all students'
Query tree root node created: root

📊 Initial Status Check:
TREE OVERVIEW: 0/1 nodes complete
PENDING: 1 need SQL, 0 need eval, 0 bad SQL
CURRENT_NODE: root
CURRENT_STATUS: needs_sql
CURRENT_INTENT: Find all students
CURRENT_NODE_CONTENT:
  - Schema linked: False (tables: none)
  - SQL generated: False (none)
  - Execution: False (none), Quality: none
OVERALL_STATUS: Processing in progress


## 2. Example: Complex Tree with Mixed Node States

Let's create a more complex tree with nodes in different states to test the prioritization logic.

In [3]:
# Clear tree and create complex scenario
root_id = await tree_manager.initialize("Complex query with multiple subqueries")

# Create child nodes using QueryNode objects and add_node
child1_id = f"child_{time.time()}_1"
child1 = QueryNode(
    nodeId=child1_id,
    intent="Filter students by grade",
    parentId=root_id
)
await tree_manager.add_node(child1, root_id)

child2_id = f"child_{time.time()}_2"
child2 = QueryNode(
    nodeId=child2_id,
    intent="Calculate average score",
    parentId=root_id
)
await tree_manager.add_node(child2, root_id)

print(f"Created complex tree:")
print(f"  Root: {root_id} - 'Complex query with multiple subqueries'")
print(f"  Child 1: {child1_id} - 'Filter students by grade'")
print(f"  Child 2: {child2_id} - 'Calculate average score'")

# Update child1 with SQL and good execution
await tree_manager.update_node_sql(child1_id, "SELECT * FROM students WHERE grade > 90")

# Create ExecutionResult object for good result
good_result = ExecutionResult(
    data=[{"id": 1, "name": "John", "grade": 95}, {"id": 2, "name": "Jane", "grade": 92}],
    rowCount=2,
    error=None
)
await tree_manager.update_node_result(child1_id, good_result, success=True)

# Store good evaluation for child1
await tree_manager.update_node_evaluation(child1_id, {
    "result_quality": "excellent",
    "answers_intent": "yes",
    "confidence_score": 0.95
})

# Update child2 with SQL but poor execution result
await tree_manager.update_node_sql(child2_id, "SELECT AVG(score) FROM students")

# Create ExecutionResult object for poor result
poor_result = ExecutionResult(
    data=[{"avg_score": None}],
    rowCount=1,
    error=None
)
await tree_manager.update_node_result(child2_id, poor_result, success=True)

# Store poor evaluation for child2  
await tree_manager.update_node_evaluation(child2_id, {
    "result_quality": "poor",
    "answers_intent": "no", 
    "confidence_score": 0.2
})

# Set current node to child2 (the problematic one)
await tree_manager.set_current_node_id(child2_id)

print(f"\n📊 Node States:")
print(f"  {child1_id}: SQL generated, executed successfully, excellent quality")
print(f"  {child2_id}: SQL generated, executed successfully, poor quality")
print(f"  {root_id}: No SQL yet (needs processing)")

# Check status
args = TaskStatusCheckerArgs(task="Analyze mixed node states")
result = await task_checker.run(args)
print(f"\n📊 Status Check with Mixed States:")
print(result)

Created complex tree:
  Root: root - 'Complex query with multiple subqueries'
  Child 1: child_1748660697.4808471_1 - 'Filter students by grade'
  Child 2: child_1748660697.4809396_2 - 'Calculate average score'

📊 Node States:
  child_1748660697.4808471_1: SQL generated, executed successfully, excellent quality
  child_1748660697.4809396_2: SQL generated, executed successfully, poor quality
  root: No SQL yet (needs processing)

📊 Status Check with Mixed States:
TREE OVERVIEW: 0/3 nodes complete
PENDING: 1 need SQL, 2 need eval, 0 bad SQL
CURRENT_NODE: child_1748660697.4809396_2
CURRENT_STATUS: needs_eval
CURRENT_INTENT: Calculate average score
CURRENT_NODE_CONTENT:
  - Schema linked: False (tables: none)
  - SQL generated: True (SELECT AVG(score) FROM students)
  - Execution: False (none), Quality: none
OVERALL_STATUS: Processing in progress


## 3. Priority Testing: Leaf Nodes First

Test that the TaskStatusChecker prioritizes leaf nodes over parent nodes in a hierarchical tree.

In [4]:
# Create a deeper tree hierarchy
root_id = await tree_manager.initialize("Main query")

# Create intermediate parent
parent_id = f"parent_{time.time()}"
parent_node = QueryNode(
    nodeId=parent_id,
    intent="Parent query",
    parentId=root_id
)
await tree_manager.add_node(parent_node, root_id)

# Create leaf nodes under parent
leaf1_id = f"leaf_{time.time()}_1"
leaf1 = QueryNode(
    nodeId=leaf1_id,
    intent="Leaf query 1",
    parentId=parent_id
)
await tree_manager.add_node(leaf1, parent_id)

leaf2_id = f"leaf_{time.time()}_2"
leaf2 = QueryNode(
    nodeId=leaf2_id,
    intent="Leaf query 2", 
    parentId=parent_id
)
await tree_manager.add_node(leaf2, parent_id)

leaf3_id = f"leaf_{time.time()}_3"
leaf3 = QueryNode(
    nodeId=leaf3_id,
    intent="Leaf query 3",
    parentId=parent_id
)
await tree_manager.add_node(leaf3, parent_id)

print(f"Created hierarchical tree:")
print(f"  Root: {root_id}")
print(f"    └── Parent: {parent_id}")
print(f"        ├── Leaf 1: {leaf1_id}")
print(f"        ├── Leaf 2: {leaf2_id}")
print(f"        └── Leaf 3: {leaf3_id}")

# Store current node before checking
current_before = await tree_manager.get_current_node_id()
print(f"\nCurrent node before: {current_before}")

# Run status check
args = TaskStatusCheckerArgs(task="Test leaf node prioritization")
result = await task_checker.run(args)
print(f"\n📊 Hierarchical Tree Status:")
print(result)

# Check which node was selected
current_after = await tree_manager.get_current_node_id()
print(f"\nCurrent node after: {current_after}")

# Verify it's a leaf node
tree_data = await tree_manager.get_tree()
selected_node = tree_data["nodes"][current_after]
has_children = len(selected_node.get("childIds", [])) > 0

print(f"\n✅ Verification:")
print(f"  Selected node is leaf (no children): {not has_children}")
print(f"  Expected: Should select a leaf node first, not parent or root")

Created hierarchical tree:
  Root: root
    └── Parent: parent_1748660699.9523351
        ├── Leaf 1: leaf_1748660699.9524083_1
        ├── Leaf 2: leaf_1748660699.9524667_2
        └── Leaf 3: leaf_1748660699.9525201_3

Current node before: root

📊 Hierarchical Tree Status:
TREE OVERVIEW: 0/5 nodes complete
PENDING: 5 need SQL, 0 need eval, 0 bad SQL
CURRENT_NODE: parent_1748660699.9523351
CURRENT_STATUS: needs_sql
CURRENT_INTENT: Parent query
CURRENT_NODE_CONTENT:
  - Schema linked: False (tables: none)
  - SQL generated: False (none)
  - Execution: False (none), Quality: none
OVERALL_STATUS: Processing in progress

Current node after: parent_1748660699.9523351

✅ Verification:
  Selected node is leaf (no children): False
  Expected: Should select a leaf node first, not parent or root


## 4. Complete Tree Scenario

Test the scenario where all nodes have been processed and the task is complete.

In [5]:
# Create a simple tree and process all nodes completely
root_id = await tree_manager.initialize("Simple complete query")

# Add one child
child_id = f"child_{time.time()}_complete"
child = QueryNode(
    nodeId=child_id,
    intent="Get student count",
    parentId=root_id
)
await tree_manager.add_node(child, root_id)

print(f"Created simple tree:")
print(f"  Root: {root_id}")
print(f"  Child: {child_id}")

# Process child completely
await tree_manager.update_node_sql(child_id, "SELECT COUNT(*) FROM students")

child_result = ExecutionResult(
    data=[{"count": 150}],
    rowCount=1,
    error=None
)
await tree_manager.update_node_result(child_id, child_result, success=True)

await tree_manager.update_node_evaluation(child_id, {
    "result_quality": "excellent",
    "answers_intent": "yes",
    "confidence_score": 0.98
})

# Process root completely 
await tree_manager.update_node_sql(root_id, "SELECT COUNT(*) FROM students")

root_result = ExecutionResult(
    data=[{"count": 150}],
    rowCount=1,
    error=None
)
await tree_manager.update_node_result(root_id, root_result, success=True)

await tree_manager.update_node_evaluation(root_id, {
    "result_quality": "excellent",
    "answers_intent": "yes",
    "confidence_score": 0.95
})

print(f"\n📊 All nodes processed with excellent quality")

# Check status
args = TaskStatusCheckerArgs(task="Check completed tree")
result = await task_checker.run(args)
print(f"\n📊 Complete Tree Status:")
print(result)
print(f"\nExpected: Should indicate task is complete")

Created simple tree:
  Root: root
  Child: child_1748660702.9071913_complete

📊 All nodes processed with excellent quality

📊 Complete Tree Status:
TREE OVERVIEW: 0/2 nodes complete
PENDING: 0 need SQL, 2 need eval, 0 bad SQL
CURRENT_NODE: child_1748660702.9071913_complete
CURRENT_STATUS: needs_eval
CURRENT_INTENT: Get student count
CURRENT_NODE_CONTENT:
  - Schema linked: False (tables: none)
  - SQL generated: True (SELECT COUNT(*) FROM students)
  - Execution: False (none), Quality: none
OVERALL_STATUS: Processing in progress

Expected: Should indicate task is complete


## 5. Error Handling and Edge Cases

Test how the TaskStatusChecker handles error conditions and edge cases.

In [6]:
# Test 1: No query tree
await memory.clear()

args = TaskStatusCheckerArgs(task="Check with no tree")
result = await task_checker.run(args)
print("📊 No Query Tree:")
print(result)
print("Expected: Error message about no query tree")

print("\n" + "="*50)

# Test 2: Empty tree (reinitialize to have something to work with)
root_id = await tree_manager.initialize("Empty query")
await memory.clear()  # Clear after init to simulate corruption

args = TaskStatusCheckerArgs(task="Check corrupted tree")
result = await task_checker.run(args)
print("\n📊 Corrupted Tree:")
print(result)

print("\n" + "="*50)

# Test 3: SQL execution failure
root_id = await tree_manager.initialize("Query with execution error")

# Update with failed execution
await tree_manager.update_node_sql(root_id, "SELECT * FROM nonexistent_table")

# Create ExecutionResult object for failed execution
failed_result = ExecutionResult(
    data=[],
    rowCount=0,
    error="Table 'nonexistent_table' doesn't exist"
)
await tree_manager.update_node_result(root_id, failed_result, success=False)

await tree_manager.update_node_evaluation(root_id, {
    "result_quality": "poor",
    "answers_intent": "no",
    "confidence_score": 0.1
})

args = TaskStatusCheckerArgs(task="Check execution failure")
result = await task_checker.run(args)
print("\n📊 Execution Failure:")
print(result)
print("Expected: Should indicate node needs improvement")

📊 No Query Tree:
STATUS: No query tree found
Expected: Error message about no query tree


📊 Corrupted Tree:
STATUS: No query tree found


📊 Execution Failure:
TREE OVERVIEW: 0/1 nodes complete
PENDING: 0 need SQL, 1 need eval, 0 bad SQL
CURRENT_NODE: root
CURRENT_STATUS: needs_eval
CURRENT_INTENT: Query with execution error
CURRENT_NODE_CONTENT:
  - Schema linked: False (tables: none)
  - SQL generated: True (SELECT * FROM nonexistent_table)
  - Execution: False (none), Quality: none
OVERALL_STATUS: Processing in progress
Expected: Should indicate node needs improvement


## 6. Tool Interface and Integration Testing

Test the TaskStatusChecker as a tool that can be used by agents and orchestrators.

In [7]:
# Test the tool interface
print("🔧 Testing TaskStatusChecker as a Tool")
print("=" * 50)

# Get the tool
tool = task_checker.get_tool()

# Check tool properties
print(f"Tool name: {tool.name}")
print(f"Tool description: {tool.description}")
print(f"Args type: {tool._args_type}")
print(f"Return type: {tool._return_type}")

# Test creating arguments
args1 = TaskStatusCheckerArgs()  # Default empty task
args2 = TaskStatusCheckerArgs(task="Check specific orchestrator request")

print(f"\nArgument Examples:")
print(f"  Default args: {args1}")
print(f"  Custom args: {args2}")

# Test the Pydantic model schema
schema = TaskStatusCheckerArgs.model_json_schema()
print(f"\nPydantic Schema:")
print(f"  Title: {schema.get('title')}")
print(f"  Properties: {list(schema.get('properties', {}).keys())}")

# Safely access task field if it exists
properties = schema.get('properties', {})
if 'task' in properties:
    task_field = properties['task']
    print(f"  Task field type: {task_field.get('type', 'unknown')}")
    print(f"  Task field description: {task_field.get('description', 'No description')}")
else:
    print(f"  Task field: Not found in schema")
    print(f"  Available fields: {list(properties.keys())}")

# Test with actual tree
root_id = await tree_manager.initialize("Tool integration test")

# Run via tool interface
tool_result = await tool.run(TaskStatusCheckerArgs(task="Orchestrator status check"))
print(f"\n📊 Tool Result:")
print(tool_result)

print(f"\n✅ Tool interface works correctly!")
print(f"   - Can be called by orchestrators")
print(f"   - Returns string status reports")
print(f"   - Uses Pydantic models for type safety")

🔧 Testing TaskStatusChecker as a Tool
Tool name: task_status_checker
Tool description: Check current state and choose the current node
Args type: <class 'task_status_checker.TaskStatusCheckerArgs'>
Return type: <class 'str'>

Argument Examples:
  Default args: goal=''
  Custom args: goal=''

Pydantic Schema:
  Title: TaskStatusCheckerArgs
  Properties: ['goal']
  Task field: Not found in schema
  Available fields: ['goal']

📊 Tool Result:
TREE OVERVIEW: 0/1 nodes complete
PENDING: 1 need SQL, 0 need eval, 0 bad SQL
CURRENT_NODE: root
CURRENT_STATUS: needs_sql
CURRENT_INTENT: Tool integration test
CURRENT_NODE_CONTENT:
  - Schema linked: False (tables: none)
  - SQL generated: False (none)
  - Execution: False (none), Quality: none
OVERALL_STATUS: Processing in progress

✅ Tool interface works correctly!
   - Can be called by orchestrators
   - Returns string status reports
   - Uses Pydantic models for type safety


## 7. Internal Analysis Functions

Explore the internal analysis functions that power the TaskStatusChecker.

In [8]:
# Create a complex tree for analysis
root_id = await tree_manager.initialize("Analysis test query")

# Create multiple nodes in different states
good_node_id = f"good_{time.time()}"
good_node = QueryNode(
    nodeId=good_node_id,
    intent="Good query",
    parentId=root_id
)
await tree_manager.add_node(good_node, root_id)

poor_node_id = f"poor_{time.time()}"
poor_node = QueryNode(
    nodeId=poor_node_id,
    intent="Poor query",
    parentId=root_id
)
await tree_manager.add_node(poor_node, root_id)

unprocessed_node_id = f"unprocessed_{time.time()}"
unprocessed_node = QueryNode(
    nodeId=unprocessed_node_id,
    intent="Unprocessed query",
    parentId=root_id
)
await tree_manager.add_node(unprocessed_node, root_id)

# Setup good node
await tree_manager.update_node_sql(good_node_id, "SELECT * FROM students LIMIT 10")

good_result = ExecutionResult(
    data=[{"id": 1, "name": "John"}],
    rowCount=1,
    error=None
)
await tree_manager.update_node_result(good_node_id, good_result, success=True)

await tree_manager.update_node_evaluation(good_node_id, {
    "result_quality": "excellent",
    "answers_intent": "yes"
})

# Setup poor node
await tree_manager.update_node_sql(poor_node_id, "SELECT * FROM missing_table")

poor_result = ExecutionResult(
    data=[],
    rowCount=0,
    error="Table not found"
)
await tree_manager.update_node_result(poor_node_id, poor_result, success=False)

await tree_manager.update_node_evaluation(poor_node_id, {
    "result_quality": "poor",
    "answers_intent": "no"
})

# Leave unprocessed_node as is

# Use TaskStatusChecker to analyze the complex tree
print("🔍 TaskStatusChecker Analysis of Complex Tree")
print("=" * 50)

# Get comprehensive analysis via the run method
args = TaskStatusCheckerArgs(task="Comprehensive analysis of mixed node states")
comprehensive_result = await task_checker.run(args)
print("Comprehensive Analysis:")
print(comprehensive_result)

# Get tree statistics directly from QueryTreeManager
print(f"\n📊 Direct Tree Statistics:")
tree_stats = await tree_manager.get_tree_stats()
print(f"  Total nodes: {tree_stats.get('total_nodes', 0)}")
print(f"  Max depth: {tree_stats.get('max_depth', 0)}")
print(f"  Leaf nodes: {tree_stats.get('leaf_count', 0)}")

# Show node types by examining tree directly
tree_data = await tree_manager.get_tree()
if tree_data:
    nodes = tree_data.get("nodes", {})
    
    good_nodes = []
    poor_nodes = []
    unprocessed_nodes = []
    
    for node_id, node_data in nodes.items():
        # Check if node has evaluation
        evaluation_key = f"node_{node_id}_evaluation"
        evaluation = await memory.get(evaluation_key)
        
        if evaluation:
            quality = evaluation.get("result_quality", "unknown")
            if quality in ["excellent", "good"]:
                good_nodes.append(node_id)
            elif quality in ["poor", "acceptable"]:
                poor_nodes.append(node_id)
        else:
            # Check if it has SQL but no evaluation
            if node_data.get("sql"):
                # Has SQL but no evaluation - might be missing evaluation
                unprocessed_nodes.append(node_id)
            else:
                # No SQL - definitely unprocessed
                unprocessed_nodes.append(node_id)
    
    print(f"\n📋 Node Classification:")
    print(f"  Good quality nodes: {len(good_nodes)} - {good_nodes}")
    print(f"  Poor quality nodes: {len(poor_nodes)} - {poor_nodes}")
    print(f"  Unprocessed nodes: {len(unprocessed_nodes)} - {unprocessed_nodes}")

print(f"\n🎯 Priority Logic Demonstration:")
if poor_nodes:
    print(f"  ✅ Priority 1: Found {len(poor_nodes)} poor nodes that need improvement")
    print(f"      TaskStatusChecker will recommend fixing these first")
elif unprocessed_nodes:
    print(f"  ✅ Priority 2: Found {len(unprocessed_nodes)} unprocessed nodes that need processing")
    print(f"      TaskStatusChecker will recommend processing these next")
else:
    print(f"  ✅ Priority 3: All nodes are good quality - task complete!")
    print(f"      TaskStatusChecker will indicate the task is finished")

🔍 TaskStatusChecker Analysis of Complex Tree
Comprehensive Analysis:
TREE OVERVIEW: 0/4 nodes complete
PENDING: 2 need SQL, 2 need eval, 0 bad SQL
CURRENT_NODE: good_1748660713.2522552
CURRENT_STATUS: needs_eval
CURRENT_INTENT: Good query
CURRENT_NODE_CONTENT:
  - Schema linked: False (tables: none)
  - SQL generated: True (SELECT * FROM students LIMIT 10)
  - Execution: False (none), Quality: none
OVERALL_STATUS: Processing in progress

📊 Direct Tree Statistics:
  Total nodes: 4
  Max depth: 0
  Leaf nodes: 0

📋 Node Classification:
  Good quality nodes: 0 - []
  Poor quality nodes: 0 - []
  Unprocessed nodes: 4 - ['root', 'good_1748660713.2522552', 'poor_1748660713.2523267', 'unprocessed_1748660713.252381']

🎯 Priority Logic Demonstration:
  ✅ Priority 2: Found 4 unprocessed nodes that need processing
      TaskStatusChecker will recommend processing these next
